In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

##OpenAI, ChatOpenAI 의 각 객체
##다른 모델을 사용할 때는 객체에서 참조하는 클래스를 변경
##llm = OpenAI(model_name="gpt-3.5-turbo-1106") --사용 안함
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,     ##Streaming 옵션 ON
    callbacks=[StreamingStdOutCallbackHandler()]
)

##동일한 질문을 각각 llm, chat에게 던짐
##llmObj = llm.predict("How many planets are there?") --사용 안함
##chatObj = chat.predict("How many planets are there?") 


Jupyter Notebook 실행 시 기본적으로 .env(현재의 가상환경) 의 환경변수를 참조함.   
OpenAI, chatOpenAI는 정확히 .env의 `OPENAI_API_KEY`를 찾아보게 되어 있음.   
(즉 환경변수 명을 정확하게 넣지 않으면 openai 구동이 불가능하다는 의미임)

이번 강의에서는 .env 내의 환경 변수로 구동했지만, `lim=OpenAI(openai_api_key="sk...")` 와 같이 생성자 형태로 넣어줄 수 있음.

이와 같이 모델을 생성자 형태로 선언할 때 `temperature`라는 매개변수를 넣어줄 수도 있음. 이 파라미터는 모델이 얼마나 창의적인지를 결정하며, 높은 값일수록 창의성이 높아지지만 무작위성 또한 높아짐

In [2]:
messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Korean"),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between Mexico and Thailand? Also, what is your name?"),
 ]

chat.predict_messages(messages)

멕시코와 태국 사이의 거리는 대략 16,000km입니다. 제 이름은 지리 전문가입니다. 어떻게 도와 드릴까요?

AIMessageChunk(content='멕시코와 태국 사이의 거리는 대략 16,000km입니다. 제 이름은 지리 전문가입니다. 어떻게 도와 드릴까요?')

In [7]:
messages_placeholder = [
     SystemMessage(content="You are a geography expert. And you only reply in {language}"),
    AIMessage(content="Ciao, mi chiamo {name}!"),
    HumanMessage(content="What is the distance between {country_a} and {country_b}? Also, what is your name?"),
]

## Message
- HumanMessage : 사용자가 챗봇에게 질의하는 Input Message이다.
- SystemMessage : 개발자가 챗봇에게 명령을 내리기 위해 사용하는 메시지이다. 주로 대화에 대한 제약사항, 가이드라인, 페르소나 등이 해당한다.
- AIMessage : AI가 사람에게 답변하는 Output Message이다.

위 코드에서는 string에 대한 predict가 아닌, messages라는 list를 넘김으로써 System/Human/AI Message를 일괄적으로 predict하였음.

다만 현재까지는 Korean, Paolo, Country 등이 하드코딩 되어 있으므로 placeholder를 통해 동적으로 바꿔 줄 필요가 있음.

## prompt
상황에 맞는 prompt를 제작하는 것이 AI 활용에서 가장 중요한 영역임.
Langchain을 포함한 LLM 영역에서 prompt를 잘 작성하는 것이 가장 중요함.
prompt끼리 결합도 가능하며, 저장/불러오기가 가능함. 변수 설정에 대한 validation도 가능

* from langchain.prompts
- PromptTemplate : template을 string으로 만듬
- ChatPromptTemplate : template을 message로 만듬

In [8]:
## Template 선언 (PromptTemplate)
template_string = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}",
)
 
## format 메소드를 사용해 template의 매개변수를 전달
prompt = template_string.format(country_a="Mexico", country_b="Thailand")

##prompt를 predict
chat.predict(prompt)


The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [9]:
##prompt 실습을 위한 객체 생성
template_message = ChatPromptTemplate.from_messages([
    ##각각의 Message를 Tuple 형태로 넣어줌
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}?"),
])

prompt_m = template_message.format_messages(
    language="Korean",
    name="Kimchi",
    country_a="South Korea",
    country_b="USA"
)

chat.predict_messages(prompt_m)

남한과 미국 사이의 거리는 대략 10,000km입니다.

AIMessageChunk(content='남한과 미국 사이의 거리는 대략 10,000km입니다.')

# LangChain Expression Language
## Output Parser
- LLM의 응답(Response)을 list 등과 같은 형태로 변형하기 위해 필요
- LLM은 항상 text로 대답하기 때문에, 원하는 data format으로 변환하는 데 사용

In [10]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",") ##strip : text의 앞뒤 공백을 잘라낸 후 , split(delicator로 parsing)
        return list(map(str.strip, items)) ## parse된 각각의 원소에도 strip을 적용

parser = CommaOutputParser()
##p.parse("Hello,how,are,you")

template_parser = ChatPromptTemplate.from_messages([
    ##각각의 Message를 Tuple 형태로 넣어줌
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of {max_items}. Do NOT reply with anything else"),
    ("human", "{question}"),
])

prompt = template_parser.format_messages(
    max_items=30,
    question="What are the colors?"
)

##chat이 predict한 결과에 parser를 적용 (parser는 text input이므로, result(AImessage)가 아닌 result.content를 넣어줘야 함)
result = chat.predict_messages(prompt)
parser.parse(result.content)

Red, blue, green, yellow, orange, purple, pink, black, white, brown, gray, turquoise, magenta, cyan, lavender, peach, maroon, navy, teal, silver, gold, beige, indigo, olive, coral, violet, mint, mustard, rust, ivory

['Red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown',
 'gray',
 'turquoise',
 'magenta',
 'cyan',
 'lavender',
 'peach',
 'maroon',
 'navy',
 'teal',
 'silver',
 'gold',
 'beige',
 'indigo',
 'olive',
 'coral',
 'violet',
 'mint',
 'mustard',
 'rust',
 'ivory']

지금까지의 내용을 정리하면,
1. ChatOpenAI 객체를 생성 (OpenAI 기반의 로봇 하나를 만든다고 생각) -> 이하 'chat'
2. chat은 string 또는 message(format)으로 predict하여 결과를 반환함.
3. message를 기반으로 predict 할 때에는 Prompt를 사용함. 이 Prompt를 동적으로(또는 간편하게) 만들기 위해 Template을 사용함
4. Template는 System/Human/AI message를 세팅한 틀이라고 이해
5. Prompt는 Template에 placeholder로 들어간 각 변수값을 지정하여 Message 또는 String을 넘겨 준다
    => 즉, Prompt를 통해 특정 parameter를 input으로 받아 Template에 담고, 이를 chat에 던져 predict함.

    * template.from_messages 로 Template 형태 설정
    * template.format_messages 로 Template 내 매개변수 설정

★★★ Chain ★★★
각 Component들 간의 function call이 일어나도록 코딩할 필요 없이, '|' 연산자로 chain을 형성 
개발자는 내부 코드를 볼 필요가 줄어듦

Chain끼리의 결합 또한 가능

Chain의 구성요소
1. Prompt (Dictionary)
    - Input : 우리가 `chain.invoke` 시 넣어주는 dictionary는 prompt Template으로 보내짐
    - Output : promptValue

2. Retriever (Single String)

3. LLM, ChatModel (Single String, list of messages or a PromptValue)
    - Input : promptValue
    - Output : ChatMessage

4. Tool (Single string/Dictionary/...)

5. OutputParser (output of LLM)
    - Input : ChatMessage

In [11]:
## 재미를 위한 템플릿 생성
poke_Template = ChatPromptTemplate.from_messages([
    ("system", "You are the pokemon master. Everything you are asked will be answered with a comma separated list of {max_items}. Do NOT reply with anything else. And reply in Korean."),
    ("human", "{question}"),
])

## 위의 코드처럼 계속해 function call이 일어나도록 코딩할 필요 없이, '|' 연산자로 chain을 형성 
chain_type = poke_Template | chat | CommaOutputParser()

## chain 실행
## chain.invoke 동안 위에서 수행했던 template.format_message를 매개변수로 자동실행함
## 즉 위의 1~5 내용을 chain 구성을 통해 한번에 수행할 수 있음.
chain_type.invoke({
    "max_items":5,
    "question":"What are the fire type pokemons?"
})


파이리, 리자드, 파이어, 블레이즈, 차모치

['파이리', '리자드', '파이어', '블레이즈', '차모치']

Chain 간의 연쇄 실습을 위해 아래 코드를 실행

In [12]:
## 요리 명을 물어봤을 때 레시피를 전달해 주는 prompt
chef_prompt = ChatPromptTemplate.from_messages({
    ("system", "Yo are a world-class internatinoal chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
})

## chain 생성 
chef_chain = chef_prompt | chat

## recipe를 물어봤을 때 적절한 채식주의 레시피로 대체해 답변하는 prompt.
## 엉뚱한 답변을 방지하기 위해, 적절한 대안이 없을 경우 없다고 대답하도록 system에 명시.
veg_chef_prompt = ChatPromptTemplate.from_messages({
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it. And If it possible, translate your response in Korean."),
    ("human", "{recipe}")
})

veg_chain = veg_chef_prompt | chat

## 두 chain을 연쇄시켜 최종 답변을 생성
## veg_chain의 input(recipe)은 chef_chain의 output이 되므로, chain을 아래와 같이 설정
final_chain = {"recipe":chef_chain} | veg_chain

## 두 번째 chain인 veg_chain의 입력값은 chef_chain의 output이 되므로,
## chain invoke 시 입력해 주는 parameter는 chef_chain의 cuisine만을 넣어줌.
final_chain.invoke({
    "cuisine":"Burger"
})




Sure! Here's a simple recipe for classic beef burgers:

Ingredients:
- 1 lb ground beef
- 1/2 teaspoon salt
- 1/4 teaspoon black pepper
- 1/4 teaspoon garlic powder
- 4 hamburger buns
- Optional toppings: lettuce, tomato, onion, cheese, ketchup, mustard, mayonnaise

Instructions:
1. In a mixing bowl, combine the ground beef, salt, pepper, and garlic powder. Mix well to combine, but be careful not to overwork the meat as it can make the burgers tough.
2. Divide the meat mixture into 4 equal portions and shape each portion into a patty, about 1/2 inch thick. Make a slight indentation in the center of each patty to prevent it from puffing up while cooking.
3. Preheat a grill or skillet over medium-high heat. If using a grill, lightly oil the grates to prevent sticking.
4. Cook the burgers for about 4-5 minutes per side for medium doneness, or until they reach your desired level of doneness. Avoid pressing down on the burgers while cooking as this can cause them to lose their juices.
5. To

AIMessageChunk(content='Here\'s a vegetarian version of the classic beef burger recipe:\n\nIngredients:\n- 1 lb plant-based ground "beef" (such as Beyond Meat or Impossible Burger)\n- 1/2 teaspoon salt\n- 1/4 teaspoon black pepper\n- 1/4 teaspoon garlic powder\n- 4 vegetarian burger buns\n- Optional toppings: lettuce, tomato, onion, cheese (use a plant-based cheese), ketchup, mustard, mayonnaise (use vegan mayo)\n\nInstructions:\n1. In a mixing bowl, combine the plant-based ground "beef", salt, pepper, and garlic powder. Mix well to combine, being careful not to overwork the mixture.\n2. Divide the mixture into 4 equal portions and shape each portion into a patty, about 1/2 inch thick. Make a slight indentation in the center of each patty.\n3. Preheat a grill or skillet over medium-high heat. If using a grill, lightly oil the grates.\n4. Cook the vegetarian burgers for about 4-5 minutes per side, or until they are cooked through.\n5. Toast the vegetarian burger buns on the grill or in 

In [30]:
## 자습을 위해 주제변경 실습
pokeType_Prompt = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 포켓몬 전문가입니다. 불 타입을 가진 포켓몬을 중복 없이 무작위로 10개 답변합니다. 답변은 comma로 구분된 list 형식이어야만 합니다. 또한, 존재하지 않는 포켓몬 이름은 반드시 답변에 포함하지 않으며 포켓몬의 이름은 반드시 공식 한국어명을 사용합니다.")
     ,("human",
      "{type} 타입의 포켓몬들을 말해줘")
])

pokeType_chain = pokeType_Prompt | chat 

pokeGen_Prompt = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 포켓몬 전문가입니다. 질문받은 각 포켓몬의 도감번호를 답변합니다. 포켓몬의 이름은 반드시 공식 한국어명을 사용합니다.")
     ,("human","{pokemon_list}")
])

pokeGen_chain = pokeGen_Prompt | chat | CommaOutputParser

poke_chain = {"pokemon_list":pokeType_chain} | pokeGen_Prompt
poke_chain.invoke({"type":"고스트"})

## 답변 자체의 정확도가 많이 떨어짐. 한국어로도 질의 입력이 가능한 것을 확인


불티니, 파이어로, 번치코, 블리니, 블리챙, 블리범, 블리자드, 블리케인, 블리키, 블리포레

ChatPromptValue(messages=[SystemMessage(content='당신은 포켓몬 전문가입니다. 질문받은 각 포켓몬의 도감번호를 답변합니다. 포켓몬의 이름은 반드시 공식 한국어명을 사용합니다.'), HumanMessage(content="content='불티니, 파이어로, 번치코, 블리니, 블리챙, 블리범, 블리자드, 블리케인, 블리키, 블리포레'")])